# Some useful reference material

I am a nerd and I use powers of two very often. I know all of the powers of two between 1..65536. I assume that you might not be as familiar with them so here is a table for your convenience. Don't forget that you have to execute the following code cell (**SHIFT+ENTER**) in order to see the actual table.

In [ ]:
# powers of two
# here are all of the "useful" powers of two between 128..262144. I say that they are "useful" 
# because most of the 1D and 2D spectra that I run have values that come from this table

print("       n     2^n")
for n in range(7, 19):
    print("{:8d}{:8d}".format(n, 2**n))

# Task 0

All of the constants are defined in the following cell. Run the code below to do all of the important background stuff before you start.

In [ ]:
# import the libraries so we can do cool math stuff and make pretty plots
import numpy as np
import matplotlib.pyplot as plt

# define a standard look for all of the plots
%matplotlib inline
font = {'size'   : 20}
plt.rc('font', **font)

# define all of our constants for these simulations
OMEGA = 10.0           # the frequency of our peak
JCOUP = 2.0            # the scalar coupling for our doublet
DECAY = 0.75           # this is T2 in seconds, which equals 1/R2
DWELL = 1 / 8000.0     # calculate the dwell time based on a sweep width of 8000.0 Hz

# Task 1

You have just acquired a nice, one-dimensional ${^1}H$ spectrum. Initially, you acquired a lot of data points and a lot of scans. More data points = better resolution, more scans = better signal-to-noise ratio. Unfortunately, this spectrum took you a very long time to acquire. The next time you synthesize this product, you decide to cut corners
* Execute the following cell (NP = 65536) and look how beautiful the FID and corresponding spectrum are.
* To save time, you decreased the number of data points that you acquired.
    * Change NP to a smaller number (still a power of 2!) and re-execute these cells to see the effect. Do this with 3-4 different values of NP. Start with big values of NP and work towards smaller values.
    * Look at how AQ (the "length" of the FID) changes each time. Can you see how this could have a huge effect on your total measurement time?

In [ ]:
NP = 65536        # number of data points recorded; must be 2^n
AQ = NP * DWELL   # the acquisition time is just the number of data points times the delay between points

# create a series of x-values that we'll get y-values for later
t = np.arange(0.00, NP * DWELL, DWELL)

# here is the calculation of the y-values based on all of the x-values above
# we're generating a complex signal so we need sin and cos terms
# both have exponential decays, both have frequencies of OMEGA, and both have couplings of JCOUP
ireal = np.cos(2 * np.pi * (OMEGA - JCOUP / 2) * t) * np.exp(-t / DECAY) \
      + np.cos(2 * np.pi * (OMEGA + JCOUP / 2) * t) * np.exp(-t / DECAY)
iimag = np.sin(2 * np.pi * (OMEGA - JCOUP / 2) * t) * np.exp(-t / DECAY) \
      + np.sin(2 * np.pi * (OMEGA + JCOUP / 2) * t) * np.exp(-t / DECAY)

# construct the complex signal for the Fourier transform
i = ireal + (iimag * 1j)

# do the Fourier transform and generate the new x-values (in Hertz)
ft_i = np.fft.fft(i)
freq = np.fft.fftfreq(NP) / DWELL

# generate the figures
fig, axs = plt.subplots(2, 1, figsize=(15,7.5))

# here is the FID
axs[0].plot(t, ireal, t, iimag)
axs[0].set_xlim((-0.02 * AQ), (AQ * 1.02))
axs[0].set_xlabel('time [s]')
axs[0].set_ylabel('intensity [arb. units]')
axs[0].set_ylim(-2.02, 2.02)
axs[0].grid(True)

# here is the spectrum
axs[1].plot(freq, ft_i.real)
axs[1].set_xlim(20.0, -20.0)
axs[1].set_xlabel('frequency [Hz]')
axs[1].set_ylabel('intensity [arb. units]')
axs[1].set_ylim(-500.0, 6500.0)
axs[1].grid(True)

# make them look pretty and display them
plt.subplots_adjust(hspace=0.5)
plt.show()

# Reflection 1

* What happened as you decreased the number of data points (NP)?
* At what point did your nice doublet turn into a singlet?
* How did the acquisition time (AQ) change as you changed the number of data points?

# Task 2

We are not always able to collect all of the data points that we need. For example, multidimensional experiments typically use fractions of the data points collected in a 1D. A quick COSY might have only 1024 x 128 data points compared to the 65536 data points that we used in our 1D!

Are we throwing all of that information away? Not really. The information is still there. If you don't believe me, just look at the FID. It still has the same oscillations. They're just shorter (smaller) than they were before. What if there was an easy way to add more data points without adding more measurement time? There is! Enter _zero-filling_ ... It does exactly what it says. We are going to fill the end of our FID with zeros. In other words, we are going to create **empty** data points and append them to our FID. All we have to do is make sure that the final size is still a power of two.

* Pick 3-4 powers of two from the table up above. Make sure that they are all bigger than 4096. Start with your smallest value of ZF and work towards the biggest.
    * Plug in your new value of ZF and execute this code
    * Watch the length of the FID
    * Look for any effects on the resulting spectrum

In [ ]:
NP = 4096        # number of data points recorded; must be 2^n
ZF = 65536        # number of data points after zero-filling; must be 2^n
AQ = ZF * DWELL  # the acquisition time is just the number of data points times the delay between points

# create a series of x-values that we'll get y-values for later
t = np.arange(0.00, NP * DWELL, DWELL)

# here is the calculation of the y-values based on all of the x-values above
# we're generating a complex signal so we need sin and cos terms
# both have exponential decays, both have frequencies of OMEGA, and both have couplings of JCOUP
ireal = np.cos(2 * np.pi * (OMEGA - JCOUP / 2) * t) * np.exp(-t / DECAY) \
      + np.cos(2 * np.pi * (OMEGA + JCOUP / 2) * t) * np.exp(-t / DECAY)
iimag = np.sin(2 * np.pi * (OMEGA - JCOUP / 2) * t) * np.exp(-t / DECAY) \
      + np.sin(2 * np.pi * (OMEGA + JCOUP / 2) * t) * np.exp(-t / DECAY)

# append zeros to the end of the acquired data
tzf = np.arange(0.00, ZF * DWELL, DWELL)
irealzf = np.append(ireal, np.zeros(ZF - NP))
iimagzf = np.append(iimag, np.zeros(ZF - NP))

# construct the complex signal for the Fourier transform
izf = irealzf + (iimagzf * 1j)

# do the Fourier transform and generate the new x-values (in Hertz)
ft_izf = np.fft.fft(izf)
freq = np.fft.fftfreq(ZF) / DWELL

# generate the figures
fig, axs = plt.subplots(2, 1, figsize=(15,7.5))

# here is the FID
axs[0].plot(tzf, irealzf, tzf, iimagzf)
axs[0].set_xlim((-0.02 * AQ), (AQ * 1.02))
axs[0].set_xlabel('time [s]')
axs[0].set_ylabel('intensity [arb. units]')
axs[0].set_ylim(-2.02, 2.02)
axs[0].grid(True)

# here is the spectrum
axs[1].plot(freq, ft_izf.real)
axs[1].set_xlim(20.0, -20.0)
axs[1].set_xlabel('frequency [Hz]')
axs[1].set_ylabel('intensity [arb. units]')
axs[1].set_ylim(-500.0, 6500.0)
axs[1].grid(True)

# make them look pretty and display them
plt.subplots_adjust(hspace=0.5)
plt.show()

# Reflection 2

* What do you think?
* How did the length of your FID change with the size of ZF?
* How did the resolution of your spectrum change with ZF?
* Did you ever recover the original intensity of the peak?
* Are there any artifacts that popped up?

# Task 3

Well, we took a crappy, low-resolution spectrum and made it look nice again but those wiggles that popped up sure are annoying. Wouldn't you agree? Look at the FID above. See how it just **ends** like that? Mathematically speaking, we took the original FID from _Task 1_ and multiplied it by a "hat" function - it has a value of _one_ for some range of values but it's _zero_ everywhere else. Well, the Fourier transform of a hat function is a sinc function. That's $f(x)=\frac{sin(x)}{x}$. Want to see what it looks like? Sure. Why not. Run this code to see what a sinc function looks like.

In [ ]:
NP = 4096        # number of data points recorded; must be 2^n
ZF = 65536        # number of data points after zero-filling; must be 2^n
AQ = ZF * DWELL  # the acquisition time is just the number of data points times the delay between points

# append zeros to the end of the acquired data
t = np.arange(0.00, ZF * DWELL, DWELL)
i = np.zeros(ZF)
i[0:NP - 1] += 1

# do the Fourier transform and generate the new x-values (in Hertz)
ft = np.fft.fft(i)
freq = np.fft.fftfreq(ZF) / DWELL

# generate the figures
fig, axs = plt.subplots(2, 1, figsize=(15,7.5))

# here is the FID
axs[0].plot(t, i)
axs[0].set_xlim((-0.02 * AQ), (AQ * 1.02))
axs[0].set_xlabel('time [s]')
axs[0].set_ylabel('intensity [arb. units]')
axs[0].set_ylim(-2.02, 2.02)
axs[0].grid(True)

# here is the spectrum
axs[1].plot(freq, ft.real)
axs[1].set_xlim(20.0, -20.0)
axs[1].set_xlabel('frequency [Hz]')
axs[1].set_ylabel('intensity [arb. units]')
# axs[1].set_ylim(-1500.0, 6500.0)
axs[1].grid(True)

# make them look pretty and display them
plt.subplots_adjust(hspace=0.5)
plt.show()

# Reflection 3

Does that look familiar? See where our problem comes from? This sinc function is killing our spectrum and we need to get rid of it. Somehow, we need to force the right edge of this hat function down to zero. That should help us out ...

# Task 4

Imagine that our FID starts on the left edge of the hat function above and goes towards the right. Call that "time zero" where t = 0. According to the hat function, at some point, our FID abruptly ends. But if we can move that right edge down to zero, then everything will change! We're going to look at three functions to do this. The first function is a simple exponential decay. That makes sense, right? I mean, the FID already has an exponential decay in it so this should be perfect. Please note that the order is important here. We have to multiply the FID by the exponential function _before_ we zero-fill.

Start with a value of LB that is small (e.g. 0.1) and gradually increase it up to a maximum of 10. For your reference, most proton spectra use an LB value of 1-3 Hz and most carbon spectra use an LB value of 1-5 Hz.

* Pick 4-5 values between 0.1..10 for LB and run the code for each one
    * What do you notice about the transition of the FID as we go from acquired data to zero-filling?
    * What do you notice about the wiggles from the sinc function?
    * Describe the height and width of the peak.

In [ ]:
NP = 4096         # number of data points recorded; must be 2^n
ZF = 65536        # number of data points after zero-filling; must be 2^n
LB = 0.1          # this is in Hertz and is similar in definition to R2 = 1/T2
AQ = ZF * DWELL   # the acquisition time is just the number of data points times the delay between points

# create a series of x-values that we'll get y-values for later
t = np.arange(0.00, NP * DWELL, DWELL)

# ireal and iimag are the real and imaginary intensities from Task 1
ireallb = ireal * np.exp(-t * LB)
iimaglb = iimag * np.exp(-t * LB)

iexp = np.exp(-t * LB)

# append zeros to the end of the acquired data
tzf = np.arange(0.00, ZF * DWELL, DWELL)
irealzf = np.append(ireallb, np.zeros(ZF - NP))
iimagzf = np.append(iimaglb, np.zeros(ZF - NP))

iexpzf = np.append(iexp, np.zeros(ZF - NP))

# construct the complex signal for the Fourier transform
izf = irealzf + (iimagzf * 1j)

# do the Fourier transform and generate the new x-values (in Hertz)
ft_izf = np.fft.fft(izf)
freq = np.fft.fftfreq(ZF) / DWELL

# generate the figures
fig, axs = plt.subplots(3, 1, figsize=(15,7.5))

# here is the exponential function
axs[0].plot(tzf, iexpzf)
axs[0].set_xlim((-0.02 * AQ), (AQ * 1.02))
axs[0].set_xlabel('time [s]')
axs[0].set_ylabel('intensity [arb.units]')
axs[0].grid(True)

# here is the FID
axs[1].plot(tzf, irealzf, tzf, iimagzf)
axs[1].set_xlim((-0.02 * AQ), (AQ * 1.02))
axs[1].set_xlabel('time [s]')
axs[1].set_ylabel('intensity [arb. units]')
axs[1].set_ylim(-2.02, 2.02)
axs[1].grid(True)

# here is the spectrum
axs[2].plot(freq, ft_izf.real)
axs[2].set_xlim(20.0, -20.0)
axs[2].set_xlabel('frequency [Hz]')
axs[2].set_ylabel('intensity [arb. units]')
axs[2].set_ylim(-500.0, 5000.0)
axs[2].grid(True)

# make them look pretty and display them
plt.subplots_adjust(hspace=0.5)
plt.show()

# Reflection 4

Well? Which values of LB offered a decent comprimise between linewidth, peak intensity, resolution, and decreased wiggles?

Can you relate the observed effect with the name (LB = line broadening) and the "apparent" $T_2$ relaxation time? Ideally, the value of LB should match the $R_2$ for your spectrum. Remember that we defined the $T_2$ value to 0.75 seconds and called it DECAY.

# Task 5

Let's try a different _window function_ to see if we can do better. For this next one, we'll try a sin or cos function. That makes sense because our signal is a series of sin and cos functions. It's also pretty nice because of the Fourier transform pairs ...
* The FT of an exponential function is a Lorentzian peak; faster exponential decay = wider peak
* The FT of a sin/cos function is a delta function (e.g. a "spike")

Bruker uses the first quarter of a cos function. It starts at 1 and stops when the function hits 0. You can shift cos function back and forth to create something that looks more like a sin function. The variable to move the function back and forth is called SSB.

* Vary SSB by picking 3-4 values between 2..8 and executing the code each time. 2 is a pure cos, anything bigger than 2 is a shifted cos/sin function with characteristics of each. Really big numbers (e.g. 100) are nearly "pure" sin functions.

In [ ]:
NP = 4096         # number of data points recorded; must be 2^n
ZF = 65536        # number of data points after zero-filling; must be 2^n
SSB = 2
AQ = ZF * DWELL   # the acquisition time is just the number of data points times the delay between points

# create a series of x-values that we'll get y-values for later
t = np.arange(0.00, NP * DWELL, DWELL)

# ireal and iimag are the real and imaginary intensities from Task 1
irealssb = ireal * np.sin((np.pi - np.pi / SSB) * (t / (DWELL * NP)) + np.pi / SSB)
iimagssb = iimag * np.sin((np.pi - np.pi / SSB) * (t / (DWELL * NP)) + np.pi / SSB)

issb = np.sin((np.pi - np.pi / SSB) * (t / (DWELL * NP)) + np.pi / SSB)

# append zeros to the end of the acquired data
tzf = np.arange(0.00, ZF * DWELL, DWELL)
irealzf = np.append(irealssb, np.zeros(ZF - NP))
iimagzf = np.append(iimagssb, np.zeros(ZF - NP))

issbzf = np.append(issb, np.zeros(ZF - NP))

# construct the complex signal for the Fourier transform
izf = irealzf + (iimagzf * 1j)

# do the Fourier transform and generate the new x-values (in Hertz)
ft_izf = np.fft.fft(izf)
freq = np.fft.fftfreq(ZF) / DWELL

# generate the figures
fig, axs = plt.subplots(3, 1, figsize=(15,7.5))

# here is the exponential function
axs[0].plot(tzf, issbzf)
axs[0].set_xlim((-0.02 * AQ), (AQ * 1.02))
axs[0].set_xlabel('time [s]')
axs[0].set_ylabel('intensity [arb.units]')
axs[0].grid(True)

# here is the FID
axs[1].plot(tzf, irealzf, tzf, iimagzf)
axs[1].set_xlim((-0.02 * AQ), (AQ * 1.02))
axs[1].set_xlabel('time [s]')
axs[1].set_ylabel('intensity [arb. units]')
axs[1].set_ylim(-2.02, 2.02)
axs[1].grid(True)

# here is the spectrum
axs[2].plot(freq, ft_izf.real)
axs[2].set_xlim(20.0, -20.0)
axs[2].set_xlabel('frequency [Hz]')
axs[2].set_ylabel('intensity [arb. units]')
axs[2].set_ylim(-500.0, 6500.0)
axs[2].grid(True)

# make them look pretty and display them
plt.subplots_adjust(hspace=0.5)
plt.show()

# Reflection 5

What do you think? Wiggles? Resolution? Line width? Peak Intensity? How cleanly does the end of the function merge into the zero-filling? Is it a smooth transition or is it a sudden change?

# Task 6

The cos function from _Task 5_ worked pretty good. Can we do better? Let's try a $cos{^2}$. Same rules as above. Pick 3-4 values of SSB that range from 2..8 and look at how the value of SSB affects the FID and spectrum.

In [ ]:
NP = 4096         # number of data points recorded; must be 2^n
ZF = 65536        # number of data points after zero-filling; must be 2^n
SSB = 2
AQ = ZF * DWELL   # the acquisition time is just the number of data points times the delay between points

# create a series of x-values that we'll get y-values for later
t = np.arange(0.00, NP * DWELL, DWELL)

# ireal and iimag are the real and imaginary intensities from Task 1
irealssb2 = ireal * np.sin((np.pi - np.pi / SSB) * (t / (DWELL * NP)) + np.pi / SSB) ** 2
iimagssb2 = iimag * np.sin((np.pi - np.pi / SSB) * (t / (DWELL * NP)) + np.pi / SSB) ** 2

issb2 = np.sin((np.pi - np.pi / SSB) * (t / (DWELL * NP)) + np.pi / SSB) ** 2

# append zeros to the end of the acquired data
tzf = np.arange(0.00, ZF * DWELL, DWELL)
irealzf = np.append(irealssb2, np.zeros(ZF - NP))
iimagzf = np.append(iimagssb2, np.zeros(ZF - NP))

issb2zf = np.append(issb2, np.zeros(ZF - NP))

# construct the complex signal for the Fourier transform
izf = irealzf + (iimagzf * 1j)

# do the Fourier transform and generate the new x-values (in Hertz)
ft_izf = np.fft.fft(izf)
freq = np.fft.fftfreq(ZF) / DWELL

# generate the figures
fig, axs = plt.subplots(3, 1, figsize=(15,7.5))

# here is the exponential function
axs[0].plot(tzf, issb2zf)
axs[0].set_xlim((-0.02 * AQ), (AQ * 1.02))
axs[0].set_xlabel('time [s]')
axs[0].set_ylabel('intensity [arb.units]')
axs[0].grid(True)

# here is the FID
axs[1].plot(tzf, irealzf, tzf, iimagzf)
axs[1].set_xlim((-0.02 * AQ), (AQ * 1.02))
axs[1].set_xlabel('time [s]')
axs[1].set_ylabel('intensity [arb. units]')
axs[1].set_ylim(-2.02, 2.02)
axs[1].grid(True)

# here is the spectrum
axs[2].plot(freq, ft_izf.real)
axs[2].set_xlim(20.0, -20.0)
axs[2].set_xlabel('frequency [Hz]')
axs[2].set_ylabel('intensity [arb. units]')
axs[2].set_ylim(-500.0, 6500.0)
axs[2].grid(True)

# make them look pretty and display them
plt.subplots_adjust(hspace=0.5)
plt.show()

# Reflection 6

I prefer a ${cos}^2(x)$ function for killing wiggles. Sometimes, that's the most important thing - like in a 2D. You can see that the resolution does suffer. You can go for a bigger value of SSB (e.g. > 3) and get a shifted cos function instead. Some of the resolution will come back, but at the expense of growing, negative lobes on either side of your peak.

This is another important mathematical feature to keep in mind: the intensity of the first data point in your FID is proportional to the area under your peak. Go back up to _Task 6_ and compare SSB = 2 versus SSB = 20. When SSB = 2, you have a cos function, the first point starts at 1, and the first point of the FID is scaled by 1 (no change!). When SSB = 20, you have a sin function, the first point starts at 0 (nearly), and the first point of the FID is scaled by 0. If the first data point is 0, then the integral must also be 0! The only way to make that happen is to have half of the peak > 0 and half of the peak < 0.